In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install ultralytics scikit-learn opencv-python-headless tensorflow joblib tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.5 MB/s eta 0:00:00


In [ ]:
import os
import cv2
import joblib
import numpy as np
from tqdm import tqdm
from ultralytics import YOLO
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# Paths in your Google Drive
train_dir = "/content/drive/MyDrive/dataset1/train"
val_dir   = "/content/drive/MyDrive/dataset1/val"

# Model save paths
model_path = "/content/drive/MyDrive/vehicle_classifier.pkl"
encoder_path = "/content/drive/MyDrive/label_encoder.pkl"

# YOLO model (for detection)
yolo_model = YOLO('yolov8n.pt')


In [ ]:
# Pre-trained ResNet50 (no top layer)
resnet = ResNet50(weights='imagenet', include_top=False, pooling='avg')

def extract_features(img_path):
    try:
        img = image.load_img(img_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        features = resnet.predict(x, verbose=0)
        return features.flatten()
    except Exception as e:
        print(f"Error processing {img_path}: {e}")
        return np.zeros((2048,))


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [6]:
def load_data(data_dir):
    X, y = [], []
    classes = os.listdir(data_dir)
    for cls in classes:
        cls_path = os.path.join(data_dir, cls)
        if not os.path.isdir(cls_path):
            continue
        for img_name in tqdm(os.listdir(cls_path), desc=f"Loading {cls}"):
            img_path = os.path.join(cls_path, img_name)
            feats = extract_features(img_path)
            X.append(feats)
            y.append(cls)
    return np.array(X), np.array(y)

# Load training data
X_train, y_train = load_data(train_dir)
# Load validation data
X_val, y_val = load_data(val_dir)


Loading emergency: 100%|██████████| 444/444 [02:44<00:00,  2.69it/s]


In [7]:
# Encode labels
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_val_enc = le.transform(y_val)

# Train Random Forest
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train_enc)

# Validate
y_pred = clf.predict(X_val)
acc = accuracy_score(y_val_enc, y_pred)
print(f"✅ Validation Accuracy: {acc*100:.2f}%")

# Save model + encoder
joblib.dump(clf, model_path)
joblib.dump(le, encoder_path)
print("✅ Model and encoder saved successfully!")


✅ Validation Accuracy: 100.00%
✅ Model and encoder saved successfully!


In [ ]:
# # Load trained model
# clf = joblib.load(model_path)
# le = joblib.load(encoder_path)

# # Input and output video paths
# input_video = "/content/drive/MyDrive/test_videos/traffic2.mp4"   # change this path to your video
# output_video = "/content/drive/MyDrive/output/processed_output.mp4"

# # Open video
# cap = cv2.VideoCapture(input_video)
# fps = cap.get(cv2.CAP_PROP_FPS)
# width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# out = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# frame_count = 0

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break
#     frame_count += 1

#     # Process every 6th frame
#     if frame_count % 6 != 0:
#         continue

#     # YOLO vehicle detection
#     results = yolo_model(frame)
#     detections = results[0].boxes

#     emergency_count = 0
#     normal_count = 0

#     for box in detections:
#         cls_id = int(box.cls)
#         cls_name = yolo_model.names[cls_id].lower()

#         # Only detect vehicles
#         if any(x in cls_name for x in ['car', 'bus', 'truck', 'motorcycle', 'ambulance']):
#             x1, y1, x2, y2 = map(int, box.xyxy[0])
#             cropped = frame[y1:y2, x1:x2]

#             if cropped.size == 0:
#                 continue

#             # Feature extraction and classification
#             img = cv2.resize(cropped, (224, 224))
#             x = np.expand_dims(img, axis=0)
#             x = preprocess_input(x)
#             features = resnet.predict(x, verbose=0).flatten().reshape(1, -1)
#             pred = clf.predict(features)
#             label = le.inverse_transform(pred)[0]

#             color = (0, 0, 255) if label == "emergency" else (0, 255, 0)
#             if label == "emergency":
#                 emergency_count += 1
#             else:
#                 normal_count += 1

#             cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
#             cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

#     print(f"Frame {frame_count}: Emergency = {emergency_count}, Normal = {normal_count}")
#     out.write(frame)

# cap.release()
# out.release()
# print("✅ Processed video saved at:", output_video)


In [9]:
# video_path = "/content/drive/MyDrive/test_video/traffic2.mp4"   # <-- your video path

# cap = cv2.VideoCapture(video_path)
# if not cap.isOpened():
#     raise Exception("❌ Could not open video. Check your path!")

# frame_count = 0

# print("🔍 Starting vehicle detection and classification...\n")

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break
#     frame_count += 1

#     # Process every 6th frame only
#     if frame_count % 6 != 0:
#         continue

#     results = yolo_model(frame, verbose=False)
#     boxes = results[0].boxes

#     emergency_count = 0
#     normal_count = 0

#     if boxes is not None and len(boxes) > 0:
#         for box in boxes:
#             cls_id = int(box.cls)
#             cls_name = yolo_model.names[cls_id].lower()

#             # Detect only relevant vehicle types
#             if any(v in cls_name for v in ['car', 'bus', 'truck', 'motorcycle', 'ambulance']):
#                 x1, y1, x2, y2 = map(int, box.xyxy[0])
#                 cropped = frame[y1:y2, x1:x2]
#                 if cropped.size == 0:
#                     continue

#                 # Feature extraction and classification
#                 img = cv2.resize(cropped, (224, 224))
#                 x = np.expand_dims(img, axis=0)
#                 x = preprocess_input(x)
#                 feats = resnet.predict(x, verbose=0).flatten().reshape(1, -1)
#                 pred  = clf.predict(feats)
#                 label = le.inverse_transform(pred)[0]

#                 if label == "emergency":
#                     emergency_count += 1
#                 else:
#                     normal_count += 1

#     # ✅ Always print even if zero detected
#     print(f"Frame {frame_count}: Emergency = {emergency_count}, Normal = {normal_count}")

# cap.release()
# print("\n✅ Finished processing video.")


In [10]:
# from google.colab.patches import cv2_imshow
# import cv2
# import numpy as np

# video_path = "/content/drive/MyDrive/test_video/traffic1.mp4"  # <-- change this to your video path

# cap = cv2.VideoCapture(video_path)
# if not cap.isOpened():
#     raise Exception("❌ Could not open video. Check your path!")

# frame_count = 0
# print("🚗 Starting vehicle detection + classification...\n")

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break
#     frame_count += 1

#     # Process every 6th frame only
#     if frame_count % 6 != 0:
#         continue

#     results = yolo_model(frame, verbose=False)
#     boxes = results[0].boxes

#     emergency_count = 0
#     normal_count = 0

#     if boxes is not None and len(boxes) > 0:
#         for box in boxes:
#             cls_id = int(box.cls)
#             cls_name = yolo_model.names[cls_id].lower()

#             # Only classify vehicles
#             if any(v in cls_name for v in ['car', 'bus', 'truck', 'motorcycle', 'ambulance']):
#                 x1, y1, x2, y2 = map(int, box.xyxy[0])
#                 cropped = frame[y1:y2, x1:x2]
#                 if cropped.size == 0:
#                     continue

#                 # Resize + extract features
#                 img = cv2.resize(cropped, (224, 224))
#                 x = np.expand_dims(img, axis=0)
#                 x = preprocess_input(x)
#                 feats = resnet.predict(x, verbose=0).flatten().reshape(1, -1)
#                 pred = clf.predict(feats)
#                 label = le.inverse_transform(pred)[0]

#                 # Counting + box color
#                 if label == "emergency":
#                     emergency_count += 1
#                     color = (0, 0, 255)  # red
#                 else:
#                     normal_count += 1
#                     color = (0, 255, 0)  # green

#                 # Draw bounding box + label
#                 cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
#                 cv2.putText(frame, label, (x1, y1 - 10),
#                             cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

#     # Print frame counts
#     print(f"Frame {frame_count}: Emergency = {emergency_count}, Normal = {normal_count}")

#     # Show the processed frame
#     display_frame = cv2.resize(frame, (640, 360))
#     cv2_imshow(display_frame)

# cap.release()
# print("\n✅ Finished processing video.")


In [ ]:
# ==================== STEP 1: Upload video from local machine ====================
from google.colab import files
import cv2
import numpy as np

# Upload a video from your computer
uploaded = files.upload()

# Get uploaded filename
for filename in uploaded.keys():
    video_path = f"/content/{filename}"  # path for OpenCV

print(f"✅ Uploaded video: {video_path}")


# ==================== STEP 2: Vehicle detection + classification ====================
from google.colab.patches import cv2_imshow

# Make sure your trained models are already loaded:
#  - yolo_model  (YOLOv8)
#  - resnet      (ResNet50 feature extractor)
#  - clf         (trained RandomForestClassifier)
#  - le          (LabelEncoder)
#  - preprocess_input (from keras.applications.resnet50)
#  If you just trained them earlier in this Colab, they are already available.

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise Exception("❌ Could not open uploaded video. Please re-upload it.")

frame_count = 0
print("🚗 Starting vehicle detection + classification...\n")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1

    # Process every 6th frame
    if frame_count % 6 != 0:
        continue

    results = yolo_model(frame, verbose=False)
    boxes = results[0].boxes

    emergency_count = 0
    normal_count = 0

    if boxes is not None and len(boxes) > 0:
        for box in boxes:
            cls_id = int(box.cls)
            cls_name = yolo_model.names[cls_id].lower()

            # Detect only vehicles
            if any(v in cls_name for v in ['car', 'bus', 'truck', 'motorcycle', 'ambulance']):
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cropped = frame[y1:y2, x1:x2]
                if cropped.size == 0:
                    continue

                # Preprocess + extract features
                img = cv2.resize(cropped, (224, 224))
                x = np.expand_dims(img, axis=0)
                x = preprocess_input(x)
                feats = resnet.predict(x, verbose=0).flatten().reshape(1, -1)
                pred = clf.predict(feats)
                label = le.inverse_transform(pred)[0]

                # Counting + box color
                if label == "emergency":
                    emergency_count += 1
                    color = (0, 0, 255)  # red
                else:
                    normal_count += 1
                    color = (0, 255, 0)  # green

                # Draw bounding box + label
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    # Print results for this frame
    print(f"Frame {frame_count}: Emergency = {emergency_count}, Normal = {normal_count}")

    # Show annotated frame
    display_frame = cv2.resize(frame, (640, 360))
    cv2_imshow(display_frame)

cap.release()
print("\n✅ Finished processing uploaded video.")


In [ ]:
# ==================== STEP 1: Upload video from local machine ====================
from google.colab import files
import cv2
import numpy as np

# Upload a video from your computer
uploaded = files.upload()

# Get uploaded filename
for filename in uploaded.keys():
    video_path = f"/content/{filename}"  # path for OpenCV

print(f"✅ Uploaded video: {video_path}")


# ==================== STEP 2: Vehicle detection + classification (count only) ====================
# Make sure your trained models are already loaded:
#  - yolo_model  (YOLOv8)
#  - resnet      (ResNet50 feature extractor)
#  - clf         (trained RandomForestClassifier)
#  - le          (LabelEncoder)
#  - preprocess_input (from keras.applications.resnet50)
# If you just trained them earlier in this Colab, they’re already available.

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise Exception("❌ Could not open uploaded video. Please re-upload it.")

frame_count = 0
print("🚗 Starting vehicle detection + classification...\n")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1

    # Process every 6th frame only
    if frame_count % 6 != 0:
        continue

    results = yolo_model(frame, verbose=False)
    boxes = results[0].boxes

    emergency_count = 0
    normal_count = 0

    if boxes is not None and len(boxes) > 0:
        for box in boxes:
            cls_id = int(box.cls)
            cls_name = yolo_model.names[cls_id].lower()

            # Only classify vehicles
            if any(v in cls_name for v in ['car', 'bus', 'truck', 'motorcycle', 'ambulance']):
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cropped = frame[y1:y2, x1:x2]
                if cropped.size == 0:
                    continue

                # Preprocess + extract features
                img = cv2.resize(cropped, (224, 224))
                x = np.expand_dims(img, axis=0)
                x = preprocess_input(x)
                feats = resnet.predict(x, verbose=0).flatten().reshape(1, -1)
                pred = clf.predict(feats)
                label = le.inverse_transform(pred)[0]

                # Counting only (no drawing)
                if label == "emergency":
                    emergency_count += 1
                else:
                    normal_count += 1

    # Print results for this frame
    print(f"Frame {frame_count}: Emergency = {emergency_count}, Normal = {normal_count}")

cap.release()
print("\n✅ Finished processing uploaded video.")
